In [1]:
import shelve

## Code Shelf

In [2]:
shelf = shelve.open('CodeShelf')

## Add or Update

In [3]:
value = """
def plot_grid(images, cols=5):
    rows = (len(images)+cols-1)//cols
    fig, axes = plt.subplots(rows, cols)

    def axis_off(ax):
        ax.set_axis_off()
        return ax

    raveled_axes = list(map(axis_off, axes.ravel()))

    for index, name in enumerate(images):
        ax = raveled_axes[index]
        image = images[name]
        ax.imshow(image) if len(
            image.shape) == 3 else ax.imshow(image, cmap='gray')
        ax.set_title(name)


for image, mask in train_prep_ds.take(2):
    mask_frequencies = np.unique(mask, return_counts=True)
    sample = {'Input Image': image, 'Input Mask: {}'.format(mask_frequencies[1]): mask}
    plot_grid(sample, cols=2)
"""

key = 'plot_image_grid'

print(shelf.get(key))
shelf[key] = value
shelf.sync()


def plot_grid(images, cols=5):
    rows = (len(images)+cols-1)//cols
    fig, axes = plt.subplots(rows, cols)

    def axis_off(ax):
        ax.set_axis_off()
        return ax

    raveled_axes = list(map(axis_off, axes.ravel()))

    for index, name in enumerate(images):
        ax = raveled_axes[index]
        image = images[name]
        ax.imshow(image) if len(
            image.shape) == 3 else ax.imshow(image, cmap='gray')
        ax.set_title(name)


for image, mask in train_prep_ds.take(2):
    mask_frequencies = np.unique(mask, return_counts=True)
    sample = {'Input Image': image, 'Input Mask: {}'.format(mask_frequencies[1]): mask}
    plot_grid(sample, cols=2)



## Enumerate

In [4]:
keys = list(shelf.keys())
print(len(keys), keys)

18 ['plot_history', 'model_summary', 'run_summary', 'tf_imports', 'basic_image_augmentations', 'image_randaugment', 'run_summary_multioutput', 'show_dataset_samples', 'draw_bb_boxes', 'dense_boxes', 'save_frames_as_gif', 'display_gifs', 'display_gifs_ipython', 'openai_virtual_display', 'plot_loss', 'color_distortion', 'display_image_dataset', 'plot_image_grid']


## Query

In [5]:
print(shelf['draw_bb_boxes'])


def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb

def bbox_to_rect(bbox, shape, color):
    "Convert bounding box to matplotlib format."
    h, w = shape
    left, top, right, bottom = bbox[0]*w, bbox[1]*h, bbox[2]*w, bbox[3]*h
    
    rect = plt.Rectangle(
        xy=(left, top), width=right-left, height=bottom-top,
        fill=False, edgecolor=color, linewidth=2)
#     print(rect)
    return rect

N_CLASSES = ds_info.features['labels'].num_classes
LABELS = ds_info.features['labels'].names
CLASS_COLORS = list(map(lambda v: rgb_to_hex(tuple(v.tolist())), np.random.choice(range(64, 255),size=[N_CLASSES, 3])))

# itr = iter(train_ds)
itr = iter(raw_train_ds)
item = next(itr)

fig = plt.imshow(item['image'])

def draw_boxes(image, fig):
    box_shape = image.shape[:2]
    objs = item['objects']
    
    for index, bbox in enumerate(objs['bbox']):
        rect = bbox_to_rect(bbox, box_shape, CLASS_COLORS[objs['label'][index]])
        fig.axes.add_patch(rect)
        fig.axes.text(